# Azure AI Agent service - Single agent

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
import json
import os
import sys
import time

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import ListSortOrder
from azure.identity import DefaultAzureCredential
from datetime import datetime, timezone, timedelta
from dotenv import load_dotenv
from openai import AzureOpenAI

In [2]:
load_dotenv(".env")

False

## Project client

In [3]:
endpoint = os.getenv("PROJECT_ENDPOINT")
model = os.getenv("MODEL_DEPLOYMENT_NAME")
credential = DefaultAzureCredential()

agents_client = AgentsClient(endpoint=endpoint, credential=credential)

## Create agent

In [5]:
agent = agents_client.create_agent(
    model=model,
    name="my-agent",
    instructions="You are helpful agent",
)
print(f"Created agent, agent ID: {agent.id}")

Created agent, agent ID: asst_5paEXYvzDLudFYrOy9MslOz8


In [6]:
prompt = "What is the best time to visit sydney?"

In [7]:
# [START create_thread]
thread = agents_client.threads.create()
# [END create_thread]
print(f"Created thread, thread ID: {thread.id}")

Created thread, thread ID: thread_ZMWOXOls5w5C3Cx3nhfZP6h5


In [8]:
# [START create_message]
message = agents_client.messages.create(thread_id=thread.id, role="user", content=prompt)
# [END create_message]
print(f"Created message, message ID: {message.id}")

Created message, message ID: msg_5BJUSsfEBhzvvy33KEj60SUO


In [9]:
# [START create_run]
run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id)
# [END create_run]
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_3EzsxlE2ekeOtJWF8bJaLWK4


In [11]:
# Poll the run as long as run status is queued or in progress
while run.status in ["queued", "in_progress", "requires_action"]:
    # Wait for a second
    time.sleep(1)
    run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"Run status: {run.status}")

    if run.status == "failed":
        print(f"Run error: {run.last_error}")

Run status: RunStatus.COMPLETED


In [12]:
run.usage

{'prompt_tokens': 39, 'completion_tokens': 283, 'total_tokens': 322, 'prompt_token_details': {'cached_tokens': 0}}

In [14]:
# [START list_messages]
messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for msg in messages:
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"{msg.role}: {last_text.text.value}")
# [END list_messages]

MessageRole.USER: What is the best time to visit sydney?
MessageRole.AGENT: The best time to visit **Sydney** depends on what you’re looking for, but generally:

### **Best Time: September to November & March to May**
- **Spring (September–November):** Pleasant temperatures (17–23°C), blooming flowers, fewer crowds, and relatively low hotel prices.
- **Autumn (March–May):** Mild weather (18–25°C), clear skies, and fewer tourists compared to summer. It’s perfect for outdoor activities and sightseeing.

---

### **Other Seasons:**
**Summer (December–February):**
- Busy tourist season, especially around Christmas/New Year
- Warm/hot weather (20–30°C)
- Beaches are lively, but accommodation prices are high and attractions are crowded

**Winter (June–August):**
- Cooler temperatures (8–17°C)
- Fewer tourists, lower hotel prices
- Good for those who enjoy quieter travel and exploring museums, galleries, and indoor attractions

---

### **Special Events**
- **New Year’s Eve (December 31):** W

## Clean Up: Delete Agent

In [ ]:
agents_client.delete_agent(agent.id)
print(f"Deleted agent, agent ID: {agent.id}")

Deleted agent, agent ID: asst_YE9x6ypds6YmyCpBxP0T1bcw
